# Dutch Ministers (DM) 1572-2004

The dataset **Dutch Ministers (DM)** is provided by the brothers van Lieburg and contains all the minister positions Dutch Reformed ministers had between 1572 and 2004. The dataset contains one row for ever carreer step of a minister and is said to **contain more up to date information** compared to [DRC](./1_1_DRC_1555-1816.ipynb) With every carreer step on a new row, this dataset contains multiple rows for every individual. For instance, Isaäc Abbema in the example below had two posts; one from 1618 to 1635 in Berkenwoude and from 1635 to 1637 in Gouda (figure 1).

![Figure 2 Dutch Ministers 1572-2004](../images/figure2.png)

*Figure 1 - Sample of Dutch Ministers 1572-2004 dataset* 

Contrary to [DRC](./1_1_DRC_1555-1816.ipynb), this dataset contains data about ministers that started their careers after 1816. What makes this dataset difficult to  work with is that individuals cannot be easily distinguished, since no unique ID is provide. Especially since over time people had the same names, individuals are not easily distinguishable. Out of the 53646 records this dataset contains, 25082 times exactly the same name is used. However, when only looking at the records that had the same name, unfeasible career paths occured. For instance the name "J. de Jong" would have had 30 positions over an unfeasible long period of time. Looking closely at “J. de Jong” it appears that this name represents multiple individuals (which is not a surprose in the Netherlands).

![Figure 3](../images/figure3.png)

*Figure 2 - Number of time a name appeared in DM* 


To integrate this dataset into the CLERUS dataset a pipeline to extract individuals out of this dataset is developed and presented in this notebook. The main idea for isolating individuals from DM is that carreer paths can be extracted by linking the rows based on the combination of the name, place where they were minister and the year that they started. This is possible since every row contains a value for the years they started and the year they left. In addition, the data contains the placename where they were minister and from where they came. Below in table 1 and example of the dataset is provided.


|...|gemeente (community/ parish) |predikant (name of minster)| ... | jaar intrede (start year) |... | jaar vertrek (end year)| ... | ... |  
|---|---|---|---|---|---|---|---|---|
|...|Hedikhuizen|Rosiere (Rosarius); Josephus van de |...| 1611 | ... | 1617 | ...| ...|
|...|Woerden|Rosiere (Rosarius); Josephus van de | ... | 1617 | ... | 1619| ...| ...|
|...|Haarlem|Rosiere (Rosarius); Josephus van de | ... | 1619 | ... | 1649| ...| ...|


Table 1 shows the records related to one carreer. This minister, i.e. Josephus van de Rosiere (Rosarius), started his carreer as minister in 1611 in Hedikhuizen after which he moved to Woerden in 1617 where he got a position until 1619. In 1619 he moved to Haarlem where he stayed until 1649 when he retired or past away. To link the various records with each other a combination between name the start year and name and end year has been created after which, by applying network creation tooling, carreer paths can be constructed by linking start_node and end_node with each other. Since the data contained some inconsistancies in the use of spaces we decided to remove all spaces from the column "predikant" which contains the name of the minister.


|...|gemeente (community/ parish) |predikant (name of minster)| ... | jaar intrede (start year) |... | jaar vertrek (end year)| ... | ... | pred_year_start | pred_year_end |
|---|---|---|---|---|---|---|---|---|---|---|
|...|Hedikhuizen|Rosiere (Rosarius); Josephus van de |...| 1611 | ... | 1617 | ...| ...|Rosiere(Rosarius);Josephusvande_1611| Rosiere(Rosarius);Josephusvande_1617|
|...|Woerden|Rosiere (Rosarius); Josephus van de | ... | 1617 | ... | 1619| ...| ...|Rosiere(Rosarius);Josephusvande_1617 | Rosiere(Rosarius);Josephusvande_1619|
|...|Haarlem|Rosiere (Rosarius); Josephus van de | ... | 1619 | ... | 1649| ...| ...|Rosiere(Rosarius);Josephusvande_1619 | Rosiere(Rosarius);Josephusvande_1649|


### Manual cleaning

Before performing this processing step, the DM dataset was cleaned. A thorough analysis scan of the dataset revealed a series of errors listed below. 
-	Information is stored in wrong column. 
-	Spaces in front of name (make it difficult to link)
-	; between name and surname is lacking, making it at a later stage difficult to split these
-	Many individuals have only one value in the field predikant, making it difficult to link these thus it is difficult to distinguish surname or name 

A round of corrections has been executed and produced an updated list. Furthermore, it contains 131 records that still needs to be checked. This however does not mean that the rest of the file does not contain any errors. This data cleaning only looked at the following issues:
-	whether “jaar intrede” has a numeric value
-	“predikant” does not start with a number
-	how many semicolons there are in field “predikant” (and if not 1 put in the list to check)
-	whether “predikant” starts with a space

Finally, every row has been given a unique ID number in the field "pid". 

In [1]:
# import the required libraries
import os
import re
import csv
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location etc.)

folderlink = '..//data//'
input_folder = 'input//'
input_pred = folderlink+input_folder+"DM_predikanten.csv"
output_folder = 'output//'
output_pred = folderlink+output_folder+"12_roles_dm.csv"

In [3]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
# ensure that the year numbers are imported as integers
jaar_vertrek_type = {'jaar vertrek': pd.Int64Dtype(), 'ind_id': pd.Int64Dtype(),'dag intrede': pd.Int64Dtype(), 'dag vertrek': pd.Int64Dtype() }
df = pd.read_csv(input_pred, sep=',', dtype= jaar_vertrek_type, encoding='utf-8')

In [5]:
# The dataset is split in two. Since the dataset is composed of rows that have information about where someone went to ('vertrek naar of vanwege') and where he or she came from ('Herkomst'), which is foremost for ministers from after 1816, and rows that have only a year when they move position a cut was made based on the field Herkomst having values NaN.
# The former is named dm_part2, the latter dm_part1.

dm_part1 = df[df['Herkomst'].isna()]

In [6]:
dm_part1.describe()

,pid,ind_id,dag intrede,jaar intrede,dag vertrek,jaar vertrek
count,23675.000000,0.0,45.0,23675.000000,39.0,23662.0
mean,27185.330475,<NA>,14.755556,1700.197973,10.307692,1713.548643
std,15617.805701,<NA>,8.842448,70.888560,8.745386,73.327799
min,1.000000,<NA>,1.0,1553.000000,1.0,165.0
25%,13782.500000,<NA>,7.0,1640.000000,2.5,1653.0
50%,27583.000000,<NA>,14.0,1699.000000,7.0,1716.0
75%,40793.500000,<NA>,23.0,1764.000000,18.0,1779.0
max,53563.000000,<NA>,31.0,2005.000000,27.0,2173.0


In [7]:
# Here all the spaces from predikant are removed
dm_part1['predikant'] = df['predikant'].str.replace(' ', '')
# Next the fields are created which are used as input for identifying carreer paths in the dataset
dm_part1['pred_year_start'] = dm_part1['predikant'].astype(str)+'_'+dm_part1['jaar intrede'].astype(str)
dm_part1['pred_year_end'] = dm_part1['predikant'].astype(str)+'_'+dm_part1['jaar vertrek'].astype(str)

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_11576\1764179776.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part1['predikant'] = df['predikant'].str.replace(' ', '')
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_11576\1764179776.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part1['pred_year_start'] = dm_part1['predikant'].astype(str)+'_'+dm_part1['jaar intrede'].astype(str)
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_11576\1764179776.py:5: SettingWithCopyWa

In [8]:
# Next the variables are set for the input of the network tool graph tool used.
tdf = dm_part1
variable_end = 'pred_year_end'
variable_start = 'pred_year_start'
variable_name = 'predikant'


In [9]:
#A function is created for the graph G tool.

def create_node_paths_dataframe(dataframe):
    # Generate the directed graph G
    G = nx.DiGraph()
    for _, row in dataframe.iterrows():
        G.add_edge(row[variable_start], row[variable_end], name=row[variable_name])

    # Find connected components and assign path ides
    paths = list(nx.connected_components(G.to_undirected()))
    path_ids = list(range(1, len(paths) + 1))
    node_path_pairs = []
    for path_id, path_nodes in zip(path_ids, paths):
        node_path_pairs.extend([(node, path_id) for node in path_nodes])

    # Next we create a DataFrame from the node-path pairs and add the a new field to the connecting paths allowing to assign individuals in the dataset.
    node_paths = pd.DataFrame(node_path_pairs, columns=[variable_start, 'individual_id'])

    # next we need to join the original DataFrame with the node-paths DataFrame to add the individual_id which distinguished individuals
    joined = pd.merge(dataframe, node_paths, left_on=variable_start, right_on=variable_start, how='left')

    return joined

In [10]:
# run the created function
result_dm_part1 = create_node_paths_dataframe(tdf)

In [11]:
result_dm_part1.head()

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden,pred_year_start,pred_year_end,individual_id
0,35685,<NA>,NH,Alkmaar,Oudkarspel,NaN,Aalburg;Johannesvan,NaN,<NA>,NaN,1743,NaN,<NA>,NaN,1777,NaN,Aalburg;Johannesvan_1743,Aalburg;Johannesvan_1777,1
1,22489,<NA>,NH,Woerden,Kalslagen,NaN,Aalst;Corneliusvan,NaN,<NA>,NaN,1717,NaN,<NA>,NaN,1751,NaN,Aalst;Corneliusvan_1717,Aalst;Corneliusvan_1751,2
2,46953,<NA>,GE,Zaltbommel,Vuren,NaN,Aalst;Gerardusvan,NaN,<NA>,NaN,1704,NaN,<NA>,NaN,1706,NaN,Aalst;Gerardusvan_1704,Aalst;Gerardusvan_1706,3
3,41854,<NA>,ZH,Schouwen-Duiveland,Sommelsdijk,NaN,Aalst;Gerardusvan,NaN,<NA>,NaN,1706,NaN,<NA>,NaN,1715,NaN,Aalst;Gerardusvan_1706,Aalst;Gerardusvan_1715,3
4,48641,<NA>,NH,Haarlem,West-Zaandam,NaN,Aalst;Gerardusvan,NaN,<NA>,NaN,1715,NaN,<NA>,NaN,1755,NaN,Aalst;Gerardusvan_1715,Aalst;Gerardusvan_1755,3


In [12]:
# create a copy of the dataset with a selection of the fields
individuals_p1 = result_dm_part1[['pid',variable_name,'individual_id']].copy()

Next we are doing the same for the second part of the dataset

In [13]:
dm_part2 = df[df['Herkomst'].notna()]

In [14]:
dm_part2['gemeente'] = dm_part2['gemeente'].str.replace(' ', '')
dm_part2['vertrek naar of vanwege'] = dm_part2['vertrek naar of vanwege'].str.replace(' ', '')

dm_part2['gemeente'] = dm_part2['gemeente'].str[:6]
dm_part2['vertrek naar of vanwege'] = dm_part2['vertrek naar of vanwege'].str[:6]


C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_11576\541579337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['gemeente'] = dm_part2['gemeente'].str.replace(' ', '')
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_11576\541579337.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['vertrek naar of vanwege'] = dm_part2['vertrek naar of vanwege'].str.replace(' ', '')
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_11576\541579337.py:4: SettingWithCopyWarning: 
A

In [15]:
dm_part2['predikant'] = dm_part2['predikant'].str.replace(' ', '')
dm_part2['pred_start'] = dm_part2['predikant'].astype(str)+'_'+dm_part2['gemeente'].astype(str)+'_'+dm_part2['jaar intrede'].astype(str)
dm_part2['pred_end'] = dm_part2['predikant'].astype(str)+'_'+dm_part2['vertrek naar of vanwege'].astype(str)+'_'+dm_part2['jaar vertrek'].astype(str)

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_11576\4125051709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['predikant'] = dm_part2['predikant'].str.replace(' ', '')
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_11576\4125051709.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['pred_start'] = dm_part2['predikant'].astype(str)+'_'+dm_part2['gemeente'].astype(str)+'_'+dm_part2['jaar intrede'].astype(str)
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_11

In [16]:
dm_part2 = dm_part2.sort_values(by='pred_start')

In [17]:
tdf = dm_part2
variable_end = 'pred_end'
variable_start = 'pred_start'
variable_name = 'predikant'


In [18]:
result_dm_part2 = create_node_paths_dataframe(tdf)

In [19]:
individuals_p2 = result_dm_part2[['pid',variable_name,'individual_id']].copy()

Now we combine the two outputs into one list. To avoid overlap between the created id fields we add the max value for the individuals in part 1 to the ids of the second part of the dataset.

In [20]:
dm_part1_max = individuals_p1['individual_id'].max()

In [21]:
individuals_p2['individual_id']= individuals_p2['individual_id'] + dm_part1_max

In [22]:
individuals = pd.concat([individuals_p1, individuals_p2], ignore_index=True)

In [23]:
# create an export of the ministers to be linked with CLERUS

In [24]:
individuals = individuals.drop('predikant', axis=1)


In [25]:
individuals.head()

,pid,individual_id
0,35685,1
1,22489,2
2,46953,3
3,41854,3
4,48641,3


In [26]:
df.head()

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden
0,35651,1,ZH,NaN,Oudewater,NaN,(Hekendorp) Brunner; L.,hulppredikant Oirschot,<NA>,NaN,1867,Lobith,<NA>,NaN,1868,NaN
1,45546,2,GE,NaN,Velp,2. Oude Jan,(Hilten van-)Matthijsen;mevrouw L.,kandidaat,2,maart,1986,Apeldoorn,23,oktober,1994,NaN
2,7179,3,NB,NaN,Breda,NaN,Aa; dr Pieter Jan Baptist Karel Simon van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,NaN
3,21505,3,GR,NaN,Hornhuizen,NaN,Aa; dr. P.J.B.K. Simon van der,kandidaat,<NA>,NaN,1862,Goutum,<NA>,NaN,1866,NaN
4,36887,3,NH,NaN,Purmerend,NaN,Aa; dr. P.J.B.K. Simon van der,Goutum,<NA>,NaN,1886,Winterswijk,<NA>,NaN,1892,NaN


In [27]:
df = pd.merge(df, individuals, left_on='pid', right_on='pid', how='left')

In [28]:
df.to_csv(output_pred, sep=';', encoding='utf-8', index=False)